In [18]:
import os,sys
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
import torchvision.transforms as transforms
import torch.nn.functional as F
import torch.utils.data as Data
import torch.utils.data as torch_data
import torchvision
from torch import optim
import string
import random

In [19]:
curr_dir = os.getcwd()
rel_path = os.path.join(curr_dir, "hw7")
model_name = '\\Name_Generator7-softmax-nnloss.pt'
hidden = 256
n = 2

In [20]:
alphabet = list(string.ascii_lowercase)
alphabet.append('*')
alphabet_size = len(alphabet)
letter_num = dict()
num_letter = dict()


for i,letter in enumerate(alphabet):
    letter_num[letter] = i
    num_letter[i] = letter

In [21]:
def oneHotEncoding(array, num_labels):
    encoded = np.zeros((np.multiply(*array.shape),num_labels),dtype=np.float32)
    encoded[np.arange(encoded.shape[0]), array.flatten()] = 1.
    encoded = encoded.reshape((*array.shape, num_labels))
    
    return encoded

In [22]:
class NamesNN(nn.Module):
    
    def __init__(self, alphabet_size, hidden,n):
        super(NamesNN,self).__init__()
        self.embedding = nn.Linear(alphabet_size,alphabet_size*5)
        self.lstm = nn.LSTM(alphabet_size*5, hidden, n,batch_first=False)
        self.hiddenToAlphabet = nn.Linear(hidden,alphabet_size)
        self.hidden = hidden
        self.dropout = nn.Dropout(0.5)
        self.softmax = nn.Softmax(dim=1)
    
    def forward(self,onehot_data,hc):
        embd = self.embedding(onehot_data)
        lstm_output, (h,c) = self.lstm(embd,hc)
        lstm_output = lstm_output.contiguous().view(lstm_output.size()[0]*lstm_output.size()[1],self.hidden)
        lstm_output = self.dropout(lstm_output)
        alphabet_sp = self.hiddenToAlphabet(lstm_output)
        alphabet_sp = F.log_softmax(alphabet_sp, dim=1)
        return alphabet_sp, (h,c)
    
    def init_hidden(self,batch_size):
        Wt = next(self.parameters()).data
        ret = Wt.new(n, max_name_length, hidden).zero_().cuda()
        return (ret,ret)

In [23]:
NNmodel = NamesNN(len(alphabet),256,2)

In [24]:
init_para = torch.load(rel_path + model_name)
PATH = "./0702-659703805-Thakur.pth"
NamesNN = torch.load(PATH)

In [27]:
def name_gen(NNmodel,first_char):
    name_list = [first_char]
    first_char = first_char.lower()
    first_char_idx = np.array([[letter_num[first_char]]])
    first_char_onehot = oneHotEncoding(first_char_idx,alphabet_size)
    first_char_onehot = torch.from_numpy(first_char_onehot)
    NNmodel.eval()
    NNmodel.cpu()
    
    hc = (torch.zeros((n,1,hidden)),torch.zeros((n,1,hidden)))
    pred,hc = NNmodel(first_char_onehot,hc)
    
    pred_num = torch.exp(pred)
    pred_topk = torch.topk(pred_num, 4)
    #print(pred_top6)
    random_prob = random.choices(pred_topk[1][0])
    pred_letter = num_letter[int(random_prob[0].numpy())]
    while pred_letter != '*':
        name_list.append(pred_letter)
        new_letter_id = np.array([[letter_num[pred_letter]]])
        new_letter_onehot = torch.from_numpy(oneHotEncoding(new_letter_id, alphabet_size))
        pred, hc = NNmodel(new_letter_onehot,hc)
        pred_num = torch.exp(pred)
        pred_topk = torch.topk(pred_num, 4)
        random_prob = random.choices(pred_topk[1][0])
        pred_letter = num_letter[int(random_prob[0].numpy())]
        
    return "".join(name_list)

In [32]:
print('Enter the starting letter for name generation:')
x = input()
names_for_a_letter = list()
while(len(names_for_a_letter)) != 20:
    new_name = name_gen(NamesNN,x.upper())
    if len(new_name) > 2:
        names_for_a_letter.append(new_name)

print(names_for_a_letter)

Enter the starting letter for name generation:
g
['Ggereviandelol', 'Ggunitevi', 'Gwesevialaneayadroansthemiva', 'Gwisaian', 'Gguassesaumie', 'Gema', 'Ggeleyorelchalerisreverlyaytcasaiaysaysoerliaiatererycoelyarelle', 'Ggironn', 'Gwisana', 'Gemahadin', 'Gwemaxsniaranolarastadilinsthannoslyarea', 'Ggaresoennnaeiaylia', 'Gem', 'Ggunc', 'Garrien', 'Genacencanndielolan', 'Ggimulaladareoreloreeazonzann', 'Ggunn', 'Gwelyarera', 'Galick']
